# XNLI Example

In this notebook, we are going to train a model for evaluating on [XNLI](https://github.com/facebookresearch/XNLI). XNLI a cross-lingual NLI task, spanning 15 different languages, with 10,000 validation and test examples per language. Notably, XNLI does not have its own training set - instead, the usual recipe is to MNLI as a training set, and is then zero-shot evaluated on NLI examples in other languages. Of course, this works best when you start with a model that has already been pretrained on a lot of multi-lingual text, such as mBERT or XLM/XLM-RoBERTa.

Hence, the tricky part about this setup is that although we have separate XNLI and MNLI tasks, we need them to all use the same task head. We will cover how to easily do this with `jiant`.

--- 

In this notebook, we will:

* Train an XLM-RoBERTa base model on MNLI
* Evaluate on XNLI-de (German) and XNLI-zh (Chinese)

## Setup

#### Install dependencies

First, we will install libraries we need for this code.

In [ ]:
%%capture
!git clone https://github.com/jiant-dev/jiant.git

In [ ]:
%%capture
# This Colab notebook already has its CUDA-runtime compatible versions of torch and torchvision installed
!sed -e /"torch==1.5.0"/d -i jiant/requirements.txt
!sed -e /"torchvision==0.6.0"/d -i jiant/requirements.txt
!pip install -r jiant/requirements.txt

#### Download data

Next, we will download MNLI and XNLI data. 

In [ ]:
%%capture
# Download/preprocess MNLI data
!wget https://raw.githubusercontent.com/huggingface/transformers/master/utils/download_glue_data.py
!python download_glue_data.py \
    --data_dir ./raw_data \
    --tasks "MNLI"
!PYTHONPATH=/content/jiant python jiant/jiant/scripts/preproc/export_glue_data.py \
    --input_base_path=./raw_data \
    --output_base_path=./tasks/ \
    --task_name_ls "mnli"

In [ ]:
%%capture
# Download XNLI data
!wget https://dl.fbaipublicfiles.com/XNLI/XNLI-1.0.zip
!unzip XNLI-1.0.zip
!mkdir -p ./tasks/data/xnli
!cp XNLI-1.0/xnli.*.jsonl ./tasks/data/xnli
!rm -rf XNLI-1.0*

## `jiant` Pipeline

In [ ]:
import sys
sys.path.insert(0, "/content/jiant")

In [ ]:
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache
import jiant.proj.main.export_model as export_model
import jiant.proj.main.scripts.configurator as configurator
import jiant.proj.main.runscript as main_runscript
import jiant.shared.caching as caching
import jiant.utils.python.io as py_io
import jiant.utils.display as display
import os

#### Task config

The first thing we need to do is to write some task config files. These JSON files tell `jiant` where to to find the data for the respective tasks, as well as the task-types and task-names.

`jiant` natively supports MNLI and XNLI task implementations. Note that we are making two different versions of the XNLI task: `xnli-de` and `xnli-zh`.

In [ ]:
LANGUAGES = ["de", "zh"]

# MNLI task configs
py_io.write_json({
    "task": "mnli",
    "name": "mnli",
    "paths": {
        "train": "/content/tasks/data/mnli/train.jsonl",
        "val": "/content/tasks/data/mnli/val.jsonl",
    },
}, path="./tasks/configs/mnli_config.json")

# XNLI task configs
for lang in LANGUAGES:
    py_io.write_json({
        "task": "xnli",
        "name": f"xnli_{lang}",
        "kwargs": {"language": lang},
        "paths": {
            "val": "/content/tasks/data/xnli/xnli.dev.jsonl",
        },
    }, path=f"./tasks/configs/xnli_{lang}_config.json")

#### Download model

Next, we will download an `xlm-roberta-base` model. This also includes the tokenizer.

In [ ]:
export_model.lookup_and_export_model(
    model_type="xlm-roberta-base",
    output_base_path="./models/xlm-roberta-base",
)

#### Tokenize and cache

With the model and data ready, we can now tokenize and cache the inputs features for our tasks. This converts the input examples to tokenized features ready to be consumed by the model, and saved them to disk in chunks.

Note that we are tokenize `train` and `val` data for MNLI, but only `val` data for XNLI, since there is no corresponding training data.

In [ ]:
# Tokenize and cache MNLI
tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
    task_config_path=f"./tasks/configs/mnli_config.json",
    model_type="xlm-roberta-base",
    model_tokenizer_path="./models/xlm-roberta-base/tokenizer",
    output_dir=f"./cache/mnli",
    phases=["train", "val"],
))

# Tokenize and cache XNLI-de, XNLI-zh
for lang in LANGUAGES:
    tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
        task_config_path=f"./tasks/configs/xnli_{lang}_config.json",
        model_type="xlm-roberta-base",
        model_tokenizer_path="./models/xlm-roberta-base/tokenizer",
        output_dir=f"./cache/xnli_{lang}",
        phases=["val"],
    ))

We can inspect the first examples of the first chunk of each task.

In [ ]:
row = caching.ChunkedFilesDataCache("./cache/mnli/train").load_chunk(0)[0]["data_row"]
print(row.input_ids)
print(row.tokens)

In [ ]:
row = caching.ChunkedFilesDataCache("./cache/xnli_de/val").load_chunk(0)[0]["data_row"]
print(row.input_ids)
print(row.tokens)

In [ ]:
row = caching.ChunkedFilesDataCache("./cache/xnli_zh/val").load_chunk(0)[0]["data_row"]
print(row.input_ids)
print(row.tokens)

#### Writing a run config

Here we are going to write what we call a `jiant_task_container_config`. This configuration file basically defines a lot of the subtleties of our training pipeline, such as what tasks we will train on, do evaluation on, batch size for each task. The new version of `jiant` leans heavily toward explicitly specifying everything, for the purpose of inspectability and leaving minimal surprises for the user, even as the cost of being more verbose.

We use a helper "Configurator" to write out a `jiant_task_container_config`, since most of our setup is pretty standard. We specify to train only on MNLI, but evaluate on MNLI, XNLI-de and XNLI-zh.

**Depending on what GPU your Colab session is assigned to, you may need to lower the train batch size.**

In [ ]:
jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
    task_config_base_path="./tasks/configs",
    task_cache_base_path="./cache",
    train_task_name_list=["mnli"],
    val_task_name_list=["mnli", "xnli_de", "xnli_zh"],
    train_batch_size=32,
    eval_batch_size=64,
    epochs=0.1,
    num_gpus=1,
).create_config()
display.show_json(jiant_run_config)

To briefly go over the major components of the `jiant_task_container_config`:

* `task_config_path_dict`: The paths to the task config files we wrote above.
* `task_cache_config_dict`: The paths to the task features caches we generated above.
* `sampler_config`: Determines how to sample from different tasks during training.
* `global_train_config`: The number of total steps and warmup steps during training.
* `task_specific_configs_dict`: Task-specific arguments for each task, such as training batch size and gradient accumulation steps.
* `taskmodels_config`: Task-model specific arguments for each task-model, including what tasks use which model.
* `metric_aggregator_config`: Determines how to weight/aggregate the metrics across multiple tasks.

**We need to make one small change to the auto-generated config**: we need to ensure that all three tasks use the same model head. Otherwise, each task will have its own task head, and the XNLI heads will be untrained.

We can make a simple change to the dictionary, setting all of them to point to an `nli_model` head, and then write out the config.

In [ ]:
jiant_run_config["taskmodels_config"]["task_to_taskmodel_map"] = {
    "mnli": "nli_model",
    "xnli_de": "nli_model",
    "xnli_zh": "nli_model",
}
os.makedirs("./run_configs/", exist_ok=True)
py_io.write_json(jiant_run_config, "./run_configs/jiant_run_config.json")

#### Start training

Finally, we can start our training run. 

Before starting training, the script also prints out the list of parameters in our model. You should notice that the only task head is the `nli_model` head.

In [ ]:
run_args = main_runscript.RunConfiguration(
    jiant_task_container_config_path="./run_configs/jiant_run_config.json",
    output_dir="./runs/run1",
    model_type="xlm-roberta-base",
    model_path="./models/xlm-roberta-base/model/xlm-roberta-base.p",
    model_config_path="./models/xlm-roberta-base/model/xlm-roberta-base.json",
    model_tokenizer_path="./models/xlm-roberta-base/tokenizer",
    learning_rate=1e-5,
    eval_every_steps=500,
    do_train=True,
    do_val=True,
    force_overwrite=True,
)

main_runscript.run_loop(run_args)

Finally, we should see the validation scores for MNLI, XNLI-de, and XNLI-zh. Given that the training data is in English, we expect to see slightly higher scores for MNLI, but the scores for XNLI-de and XNLI-zh are still decent!